# Specialist Certificate in Data Analytics Essentials Assignment

## Chapter 2: Merging Tables Stored in the Chinook Database

## Imports

In [1]:
%cd ..

/home/michael/Documents/python_projects/UCDPA_Michael_Sandilands


In [2]:
import pandas as pd
import sqlalchemy as sql

## Import Data

The `sqlalchemy` module allows me to connect to the database through python.

### Connecting to Database

In [3]:
engine = sql.create_engine("sqlite:///00_Data/Chinook_Sqlite.sqlite")

conn = engine.connect()

### Get Table Names

In [4]:
inspector = sql.inspect(conn)

inspector.get_schema_names()

['main']

In [5]:
table_names = inspector.get_table_names('main')

print(table_names)

['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


### Read the Data

I'll store all the the tables in a dictionary then use that dictionary to explore their contents. The key will be the name of the table, the value will be the table itself.

In [6]:
data_dict = {}
for table in table_names:
    data_dict[table] = pd.read_sql(f'SELECT * FROM {table}', con=conn) 

### Close Connection

In [7]:
conn.close()

## Combining Customer Segmentation Features

The relationships between the Chinook tables:


![Chinook Table Relationships](../00_Images/chinook-diagram.png)

We have to think about which features will be important in answering our business question. I'll use an iterator to display each table.

In [8]:
table_itr = iter(table_names)

In [9]:
data_dict[next(table_itr)].head()

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3


 From the 'Album' table:
- The 'Title' column may be useful.
- The 'ArtistId' column will be needed to add the 'Artist' table to the full customer segmentation data frame.

In [10]:
data_dict[next(table_itr)].head()

,ArtistId,Name
0,1,AC/DC
1,2,Accept
2,3,Aerosmith
3,4,Alanis Morissette
4,5,Alice In Chains


From the 'Artist' table:
- The 'Name' column may be useful.

In [11]:
data_dict[next(table_itr)].head()

,CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,Fax,Email,SupportRepId
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
2,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
3,4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
4,5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


From the 'Customer' table:
- For this iteration of customer segmentation I'm not going to include geographic features. These can be explored down the road, but for now I'm going to start off simple and focus on product features in order to segment customers.

In [12]:
data_dict[next(table_itr)].head()

,EmployeeId,LastName,FirstName,Title,ReportsTo,BirthDate,HireDate,Address,City,State,Country,PostalCode,Phone,Fax,Email
0,1,Adams,Andrew,General Manager,NaN,1962-02-18 00:00:00,2002-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
1,2,Edwards,Nancy,Sales Manager,1.0,1958-12-08 00:00:00,2002-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
2,3,Peacock,Jane,Sales Support Agent,2.0,1973-08-29 00:00:00,2002-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
3,4,Park,Margaret,Sales Support Agent,2.0,1947-09-19 00:00:00,2003-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
4,5,Johnson,Steve,Sales Support Agent,2.0,1965-03-03 00:00:00,2003-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com


From the 'Employee' table:
- I don't think sales rep. information is important in segmenting the customers.

In [13]:
data_dict[next(table_itr)].head()

,GenreId,Name
0,1,Rock
1,2,Jazz
2,3,Metal
3,4,Alternative & Punk
4,5,Rock And Roll


From the 'Genre' table:
- The 'Name' column may be useful.

In [14]:
data_dict[next(table_itr)].head()

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,1,2,2009-01-01 00:00:00,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
1,2,4,2009-01-02 00:00:00,Ullevålsveien 14,Oslo,None,Norway,0171,3.96
2,3,8,2009-01-03 00:00:00,Grétrystraat 63,Brussels,None,Belgium,1000,5.94
3,4,14,2009-01-06 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
4,5,23,2009-01-11 00:00:00,69 Salem Street,Boston,MA,USA,2113,13.86


From the 'Invoice' table:
- We want to pull in the 'CustomerId' column because we have to have a way to relate the 'InvoiceLine' to a sepecific customer.

In [15]:
data_dict[next(table_itr)].head()

,InvoiceLineId,InvoiceId,TrackId,UnitPrice,Quantity
0,1,1,2,0.99,1
1,2,1,4,0.99,1
2,3,2,6,0.99,1
3,4,2,8,0.99,1
4,5,2,10,0.99,1


From the 'InvoiceLine' table:
- I'll use this table as the "bottom" level for the merging process. 

In [16]:
data_dict[next(table_itr)].head()

,MediaTypeId,Name
0,1,MPEG audio file
1,2,Protected AAC audio file
2,3,Protected MPEG-4 video file
3,4,Purchased AAC audio file
4,5,AAC audio file


In [17]:
data_dict[next(table_itr)].head()

,PlaylistId,Name
0,1,Music
1,2,Movies
2,3,TV Shows
3,4,Audiobooks
4,5,90’s Music


In [18]:
data_dict[next(table_itr)].head()

,PlaylistId,TrackId
0,1,3402
1,1,3389
2,1,3390
3,1,3391
4,1,3392


From the 'MediaType', 'Playlist' and 'PlaylistTrack' tables:
- I don't think this information will be important in segmenting the customers. 
- We don't care about what format the product is distributed in, the purchase of the product itself is what matters to us here.

In [19]:
data_dict[next(table_itr)].head()

,TrackId,Name,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,UnitPrice
0,1,For Those About To Rock (We Salute You),1,1,1,"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99
1,2,Balls to the Wall,2,2,1,None,342562,5510424,0.99
2,3,Fast As a Shark,3,2,1,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99
3,4,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99
4,5,Princess of the Dawn,3,2,1,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99


From the 'Track' table:
- The 'Name' column may be useful.
- The 'Composer' column may be useful.
- The 'AlbumId' and 'GenreId' columns will be needed to merge the 'Album' and 'Genre' tables respectively to our full customer segmentation data frame.

The 'Name' column is common in a lot of tables. This column will have to be renamed as each table is joined such that it makes descriptive sense in a fully joined table.

In [20]:
joined_df = pd.DataFrame(data_dict['InvoiceLine']) \
    .merge(
        right    = data_dict['Track'] \
            # The 'UnitPrice' column is redundant as it's already in the 'InvoiceLine' table
            .drop('UnitPrice', axis=1) \
            .rename(columns = dict(Name = 'TrackName')),
        how      = 'left',
        left_on  = 'TrackId',
        right_on = 'TrackId'
    ) \
    .merge(
        right    = data_dict['Genre'] \
            .rename(columns = dict(Name = 'GenreName')),
        how      = 'left',
        left_on  = 'GenreId',
        right_on = 'GenreId'
    ) \
    .merge(
        right    = data_dict['Album'] \
            # The 'Title' column isn't very descriptive so I'm going to prepend 'Album' to the column name
            .rename(columns = dict(Title = 'AlbumTitle')),
        how      = 'left',
        left_on  = 'AlbumId',
        right_on = 'AlbumId'
    ) \
    .merge(
        right    = data_dict['Artist'] \
            .rename(columns = dict(Name = 'ArtistName')),
        how      = 'left',
        left_on  = 'ArtistId',
        right_on = 'ArtistId'
    ) \
    .merge(
        # We only want the 'InvoiceId' and 'CustomerId' columns.
        right    = data_dict['Invoice'][['InvoiceId', 'CustomerId']],
        how      = 'left',
        left_on  = 'InvoiceId',
        right_on = 'InvoiceId'
    )

joined_df.head()

# Is 'UnitPrice' column from the 'Track' table redundent? 
# UnitPrice_Logical_Series = joined_df['UnitPrice_x'] == joined_df['UnitPrice_y']
# UnitPrice_Logical_Series.sum() # Yes DONE: edit 'Track' merge to drop 'UnitPrice' column.

,InvoiceLineId,InvoiceId,TrackId,UnitPrice,Quantity,TrackName,AlbumId,MediaTypeId,GenreId,Composer,Milliseconds,Bytes,GenreName,AlbumTitle,ArtistId,ArtistName,CustomerId
0,1,1,2,0.99,1,Balls to the Wall,2,2,1,None,342562,5510424,Rock,Balls to the Wall,2,Accept,2
1,2,1,4,0.99,1,Restless and Wild,3,2,1,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,Rock,Restless and Wild,2,Accept,2
2,3,2,6,0.99,1,Put The Finger On You,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",205662,6713451,Rock,For Those About To Rock We Salute You,1,AC/DC,4
3,4,2,8,0.99,1,Inject The Venom,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",210834,6852860,Rock,For Those About To Rock We Salute You,1,AC/DC,4
4,5,2,10,0.99,1,Evil Walks,1,1,1,"Angus Young, Malcolm Young, Brian Johnson",263497,8611245,Rock,For Those About To Rock We Salute You,1,AC/DC,4


## Dropping Unnecessary Columns

We want to drop the columns that end with 'Id' except for the columns that start with 'Invoice' or 'Customer' 

In [21]:
final_joined_df = joined_df.loc[:, joined_df.columns.str.contains('^Invoice|^Customer|(?<!Id)$')]

final_joined_df.head()

,InvoiceLineId,InvoiceId,UnitPrice,Quantity,TrackName,Composer,Milliseconds,Bytes,GenreName,AlbumTitle,ArtistName,CustomerId
0,1,1,0.99,1,Balls to the Wall,None,342562,5510424,Rock,Balls to the Wall,Accept,2
1,2,1,0.99,1,Restless and Wild,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,Rock,Restless and Wild,Accept,2
2,3,2,0.99,1,Put The Finger On You,"Angus Young, Malcolm Young, Brian Johnson",205662,6713451,Rock,For Those About To Rock We Salute You,AC/DC,4
3,4,2,0.99,1,Inject The Venom,"Angus Young, Malcolm Young, Brian Johnson",210834,6852860,Rock,For Those About To Rock We Salute You,AC/DC,4
4,5,2,0.99,1,Evil Walks,"Angus Young, Malcolm Young, Brian Johnson",263497,8611245,Rock,For Those About To Rock We Salute You,AC/DC,4


Let's break down this regex:
- '^Invoice': starts with 'Invoice'.
- '^Customer': starts with 'Customer'.
- '(?<!Id)$': Negative look-behind. What immediately precedes the end of the string is not 'Id'.
- '|': Either or.
- In plain english: Keep the columns that starts with 'Invoice' or starts with 'Customer' or does not end with 'Id'.

## Relocating Columns

I personally prefer to have the 'Id' columns at the front of the data frame

In [22]:
cols_list = final_joined_df.columns.tolist()

cols_to_front = [col for col in cols_list if 'Id' in col]
cols_remaining = [col for col in cols_list if col not in cols_to_front]

invoice_lines_df = final_joined_df[[*cols_to_front, *cols_remaining]]

invoice_lines_df.head()

,InvoiceLineId,InvoiceId,CustomerId,UnitPrice,Quantity,TrackName,Composer,Milliseconds,Bytes,GenreName,AlbumTitle,ArtistName
0,1,1,2,0.99,1,Balls to the Wall,None,342562,5510424,Rock,Balls to the Wall,Accept
1,2,1,2,0.99,1,Restless and Wild,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,Rock,Restless and Wild,Accept
2,3,2,4,0.99,1,Put The Finger On You,"Angus Young, Malcolm Young, Brian Johnson",205662,6713451,Rock,For Those About To Rock We Salute You,AC/DC
3,4,2,4,0.99,1,Inject The Venom,"Angus Young, Malcolm Young, Brian Johnson",210834,6852860,Rock,For Those About To Rock We Salute You,AC/DC
4,5,2,4,0.99,1,Evil Walks,"Angus Young, Malcolm Young, Brian Johnson",263497,8611245,Rock,For Those About To Rock We Salute You,AC/DC


In [23]:
print(invoice_lines_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2240 entries, 0 to 2239
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   InvoiceLineId  2240 non-null   int64  
 1   InvoiceId      2240 non-null   int64  
 2   CustomerId     2240 non-null   int64  
 3   UnitPrice      2240 non-null   float64
 4   Quantity       2240 non-null   int64  
 5   TrackName      2240 non-null   object 
 6   Composer       1644 non-null   object 
 7   Milliseconds   2240 non-null   int64  
 8   Bytes          2240 non-null   int64  
 9   GenreName      2240 non-null   object 
 10  AlbumTitle     2240 non-null   object 
 11  ArtistName     2240 non-null   object 
dtypes: float64(1), int64(6), object(5)
memory usage: 227.5+ KB
None


## Writing to CSV

In [24]:
invoice_lines_df.to_csv('./00_Data/invoice_lines.csv', index=False)